# Финальный проект по ООП

In [3]:
import pandas as pd

# Загрузка данных из файлов
file_paths = {
    "nafld1": "nafld1.csv",
    "nafld2": "nafld2.csv",
    "nwtco": "nwtco.csv"
}

# Чтение каждого файла
dataframes = {name: pd.read_csv(path) for name, path in file_paths.items()}

# Получение информации о каждом датасете
dataset_info = {name: {"shape": df.shape, "columns": df.columns.tolist(), "head": df.head()} for name, df in dataframes.items()}
dataset_info


{'nafld1': {'shape': (17549, 10),
  'columns': ['Unnamed: 0',
   'id',
   'age',
   'male',
   'weight',
   'height',
   'bmi',
   'case.id',
   'futime',
   'status'],
  'head':    Unnamed: 0  id  age  male  weight  height        bmi  case.id  futime  \
  0        3631   1   57     0    60.0   163.0  22.690939  10630.0    6261   
  1        8458   2   67     0    70.4   168.0  24.884028  14817.0     624   
  2        6298   3   53     1   105.8   186.0  30.453537      3.0    1783   
  3       15398   4   56     1   109.3   170.0  37.830100   6628.0    3143   
  4       13261   5   68     1     NaN     NaN        NaN   1871.0    1836   
  
     status  
  0       0  
  1       0  
  2       0  
  3       0  
  4       1  },
 'nafld2': {'shape': (400123, 5),
  'columns': ['Unnamed: 0', 'id', 'days', 'test', 'value'],
  'head':    Unnamed: 0  id  days  test  value
  0      135077   1  -459   hdl   75.0
  1      313143   1  -459  chol   75.0
  2      135078   1   183   hdl   64.0
  3     

#### nafld1.csv
* Размер: 17549 строк и 10 столбцов.\
Столбцы:
* Unnamed: 0, id, age, male, weight, height, bmi, case.id, futime, status.

#### nafld2.csv
* Размер: 400123 строки и 5 столбцов.\
Столбцы:
* Unnamed: 0, id, days, test, value.

#### nwtco.csv
* Размер: 4028 строк и 10 столбцов.\
Столбцы:
* Unnamed: 0, seqno, instit, histol, stage, study, rel, edrel, age, in.subcohort.

---

### Удалить столбцы Unnamed: 0

In [4]:
# Удаление столбца 'Unnamed: 0', если он является индексом
for name, df in dataframes.items():
    if 'Unnamed: 0' in df.columns:
        dataframes[name] = df.drop(columns=['Unnamed: 0'])

# Проверим обновлённые столбцы в каждом датасете
updated_columns = {name: df.columns.tolist() for name, df in dataframes.items()}
updated_columns

{'nafld1': ['id',
  'age',
  'male',
  'weight',
  'height',
  'bmi',
  'case.id',
  'futime',
  'status'],
 'nafld2': ['id', 'days', 'test', 'value'],
 'nwtco': ['seqno',
  'instit',
  'histol',
  'stage',
  'study',
  'rel',
  'edrel',
  'age',
  'in.subcohort']}

Столбец Unnamed: 0 был успешно удалён из всех датасетов. Теперь их структура выглядит так:

nafld1
* Столбцы: id, age, male, weight, height, bmi, case.id, futime, status.

nafld2
* Столбцы: id, days, test, value.

nwtco
* Столбцы: seqno, instit, histol, stage, study, rel, edrel, age, in.subcohort.

---

### Изучим ключевые столбцы. 

Основные шаги:
1. Проверим, есть ли дублирующиеся значения в столбцах id, case.id, seqno.
2. Определим, есть ли пересечения между идентификаторами из разных датасетов.
3. Посмотрим распределение данных в этих столбцах.

In [5]:
# Проверка уникальности и пересечений ключевых столбцов

# Уникальные значения в ключевых столбцах
key_columns_analysis = {
    "nafld1_id_unique": dataframes["nafld1"]["id"].nunique(),
    "nafld2_id_unique": dataframes["nafld2"]["id"].nunique(),
    "nwtco_seqno_unique": dataframes["nwtco"]["seqno"].nunique(),
}

# Проверка пересечений между идентификаторами в датасетах
nafld1_ids = set(dataframes["nafld1"]["id"])
nafld2_ids = set(dataframes["nafld2"]["id"])
nwtco_seqnos = set(dataframes["nwtco"]["seqno"])

key_columns_analysis["nafld1_nafld2_overlap"] = len(nafld1_ids & nafld2_ids)
key_columns_analysis["nafld1_nwtco_overlap"] = len(nafld1_ids & nwtco_seqnos)
key_columns_analysis["nafld2_nwtco_overlap"] = len(nafld2_ids & nwtco_seqnos)

key_columns_analysis

{'nafld1_id_unique': 17549,
 'nafld2_id_unique': 15666,
 'nwtco_seqno_unique': 4028,
 'nafld1_nafld2_overlap': 15666,
 'nafld1_nwtco_overlap': 4023,
 'nafld2_nwtco_overlap': 3575}

***Результаты анализа ключевых столбцов:***

1. Уникальные значения:
* В nafld1 столбец id содержит 17549 уникальных значений.
* В nafld2 столбец id содержит 15666 уникальных значений.
* В nwtco столбец seqno содержит 4028 уникальных значений.

2. Пересечения между датасетами:
* Между nafld1 и nafld2 есть 15666 общих идентификаторов (id).
* Между nafld1 и nwtco есть 4023 общих идентификатора (id и seqno).
* Между nafld2 и nwtco есть 3575 общих идентификаторов.
---
Что это значит:
* nafld1 и nafld2 хорошо пересекаются по столбцу id. Это позволяет связать данные из этих датасетов.
* nwtco частично пересекается с обоими другими датасетами. Это стоит учесть при объединении данных.